# Searching and accessing data using the DL Enterprise Accelerator

In this notebook you will access data using the Scenes API. Streamlined data access is core to the DL Enterprise Accelerator. To demonstrate the basics of exploring and accessing data we will be constructing a cloud-free Sentinel-2 L2A NDVI composite over an area in the corn-belt of the United States (Iowa specifically). 

We will use the following DL API's in this exercise:
- [Scenes](https://docs.descarteslabs.com/descarteslabs/scenes/readme.html) - Query for and access imagery over our AOI
- [Geo](https://docs.descarteslabs.com/descarteslabs/geo/readme.html) - Generate a DLTile aoi


We will use the following external Python packages:
- [rasterio](https://rasterio.readthedocs.io/en/latest/index.html) - CRS aware plotting
- [affine](https://github.com/rasterio/affine) - Translate metadata returned from Scenes into usable Affine transform for plotting
- [matplotlib.pyplot](https://matplotlib.org/3.5.0/api/_as_gen/matplotlib.pyplot.html) - Plot imagery
- [numpy](https://numpy.org/doc/stable/index.html) - Array/imagery operations and manipulations

In [ ]:
import descarteslabs as dl
from rasterio.plot import show
from affine import Affine
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
dl.config.get_settings().current_env

## Creating an AOI

We start off by creating an area of interest using the `dl.geo` submodule. We can create a DLTile for a specific latitude and longitude. [A DLTile](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.DLTile) is a part of the tiling system used by DL to split up the surface of the Earth into manageable chunks for analysis. When creating a DLTile we must specify a resolution (in meters), a tilesize (in number of pixels at the given resolution), and a padding if desired (in pixels).

In [ ]:
tile = dl.geo.DLTile.from_latlon(35.8052, -91.5531, 10, 1024, 0)

Let's look at the tile object:

In [ ]:
tile

The DLTile not only contains information about the resolution, tilesize, and pad but also has an associated coordinate reference system (crs), bounds, geometry, geotransform, and a proj4 string. All DLTiles are created using a UTM (universal transverse mercator) crs. If you would like to create an AOI with a custom geometry and crs then you can specify [an AOI instead](https://docs.descarteslabs.com/descarteslabs/geo/readme.html#descarteslabs.geo.AOI).

## Searching for imagery

With our DLTile ready we can now search for and access imagery using Scenes. We start by searching the available data products in the DL Data Catalog. We do this using `scenes.search_products`. You can filter this search by specifying several arguments. For more information please see [the docs here](https://docs.descarteslabs.com/descarteslabs/scenes/docs/search.html#descarteslabs.scenes._search.search_products).

In [ ]:
result = dl.scenes.search_products(text="Sentinel-2")

In [ ]:
result

We get back two results from our search. One data product for Sentinel-1 and another for Sentinel-2. We will be using the Sentinel-2 L2A data product. To search for data in that product we need to use the `scenes.search` function. There are a few other parameters we should specify to filter what imagery we would like to access when using that function:

- product - A DL Catalog product string specifying which unique product we would like to access data from
- start_datetime - The beginning of the date range we would like to find imagery in
- end_datetime - The end of the date range we would like to find imagery in

In [ ]:
product = "esa:sentinel-2:l2a:v1"
start_datetime = "2019-03-01"
end_datetime = "2019-10-01"

In [ ]:
print(product)

We can plug in our AOI, product, start and end datetimes to the `dl.scenes.search()` function to find imagery meeting the specified arguments. We also specify `limit=None` and `cloud_fraction=0.25` to allow us to access as many scenes as are available that have an overall cloud fraction less than or equal to 25%. Cloud fraction here means the percent of an image that is covered by clouds.

In [ ]:
scenes, ctx = dl.scenes.search(
    tile,
    products=product,
    start_datetime=start_datetime,
    end_datetime=end_datetime,
    limit=None,
    cloud_fraction=0.25
)

We get a `SceneCollection` and `GeoContext` back from the `dl.scenes.search()` call. The `SceneCollection` object contains metadata about the scenes we queried and methods for accessing those scenes. The `GeoContext` has information about the CRS, resolution, bounds, and shape associated with the AOI we specified and the underlying imagery we queried. The `GeoContext` is used to specify what scale, coordinate system, and over what area we want our imagery to be resampled, transformed, and clipped to. By default a `dl.scenes.search()` call will return a `GeoContext` with the native resolution and coordinate system of the imagery we are accessing. You can adjust these components of the `GeoContext` using the `assign()` method. For this notebook we will use the native resolution and CRS.

In [ ]:
scenes

In [ ]:
ctx

A `SceneCollection` contains metadata about the query you made using `dl.scenes.search()` as well as a series of `Scenes` objects each with their own metadata and methods. We can access this metadata using `SceneCollection.properties` which will list the metadata/properties for each fo the collections individual `Scenes`. This is useful for extracting information like a list of acquisition dates for all the images we queried.

In [ ]:
dates = [key for key, scene in scenes.groupby(lambda x: x.properties.date.strftime("%Y-%m-%d"))]

In [ ]:
print(dates)

A `SceneCollection` can also be filtered by the various properties of its constituent `Scenes`. This is done using `SceneCollection.filter()` and either a predicate string of the form "properties.property" or using a lambda function (lambda x: x.properties.property). `SceneCollection`s can also be grouped based on properties using the `SceneCollection.groupby()` method. We won't be filtering or grouping the scenes in our collection for the sake of simplicity.

## Accessing imagery

Now that we have queried the available imagery we can access the actual raster data.

A `Scene` has two methods for pulling an image locally:
- `.ndarray()` - Pulls the image data into a 3D numpy.ndarray of shape *(n bands, xs, ys)*
- `.download()` - Downloads the image data down into a .geotiff at a specified filepath

A `SceneCollection` has a variety of methods that allow you to pull imagery into your local environment:
- `.stack()` - Pulls each individual image in parallel and stacks them all into a 4D array of shape *(n images, n bands, xs, ys)*
- `.mosaic()` - Pulls all images into a single 3D array where the values are populated using the "top-most" image in the collection (specified by the last image in the `SceneCollection`)
- `.download()` - Downloads each image into a .geotiff in parallel
- `download_modaic()` - Downloads a single mosaic into a .geotiff using the same logic as `.mosaic()`

We will be using `.stack()` to create a 4D array of our queried imagery. We need to specify a few arguments for this method. We specify which bands we want to access (in this case the red and near infrared bands). We also want the cloud mask band to remove any cloudy pixels from our final composite. We specify that we want to mosaic images from the same aquisition day using the `flatten` kwarg. This will mean that the output array will have shape *(time/day, n bands, xs, ys)*. We also want to have the `.stack()` call to return some raster metadata that we can use for plotting our raster data cleanly. The final thing we specify is our "scaling". For this example we want reflectance values between 0-1.0. To do this we set `scaling="physical"`. For more info on scaling parameters please see [here](https://docs.descarteslabs.com/descarteslabs/scenes/docs/scene.html#descarteslabs.scenes.scene.Scene.scaling_parameters).

In [ ]:
bands = ["red", "nir"]

In [ ]:
stack, meta = scenes.stack(
    bands+["cloud_mask"],
    ctx,
    flatten=lambda x: x.properties.date.strftime("%Y-%m-%d"),
    raster_info=True,
    scaling="physical"
)

In [ ]:
stack.shape

With our imagery now pulled locally into an ndarray we can proceed to mask out any cloud pixels and compute NDVI (Normalized Difference Vegetation Index; a representation of crop health). NDVI can be computed using
**(nir-red)/(nir + red)**.

In [ ]:
stack.mask = (stack.mask) | np.repeat((stack[:,-1].data==1)[:, np.newaxis], stack.shape[1], axis=1)

In [ ]:
red = stack[:,0]
nir = stack[:,1]

In [ ]:
ndvi = (nir - red)/(nir + red)

In [ ]:
stack=None

In [ ]:
ndvi.shape

We now build our maximum NDVI composite by using the `.max()` method on our stack.

In [ ]:
max_ndvi = ndvi.max(axis=0)

As mentioned before we can pull some useful metadata from the returned raster_info to use for plotting with rasterio.plot.show.

In [ ]:
epsg_str = f"EPSG:{meta[0]['coordinateSystem']['epsg']}"
af_transform = Affine.from_gdal(*meta[0]["geoTransform"])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10,10))

show(
    max_ndvi,
    transform=af_transform,
    ax=ax
)
ax.axis("off")

## ToDo:
Add link to new Catalog - https://packaged-analytics.dev.aws.descarteslabs.com/datasets/esa:sentinel-2:l2a:v1